## 猜字遊戲Hangman

In [90]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        secret_word 猜的字、
        guesser 模型
        max_mistakes 錯誤的次數預設為8
        verbose 預設為 True 表示互動性猜字
    """
    validLetters = 'abcdefghijklmnopqrstuvwxyz'
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    
    print("『Hangman』，hints：", ' '.join(mask), 'character : ', len(secret_word))
    mistakes = 0
    
    while mistakes < max_mistakes:
        
        print(max_mistakes-mistakes, "turns left")
        guess = guesser(mask, guessed, **guesser_args)
        
        if guess in guessed:
            print('Try Again！,used')
            mistakes += 1
        elif guess not in validLetters:
            print("Try Again！,Not valid character")
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                print('Guess the word:，', ' '.join(mask))
            else: 
                mistakes += 1

        if '_' not in mask:
            return mistakes
        
    print('『You loose』，The Ans :', secret_word) 
    print("You loose, You let a kind man die")
    print("  --------  ")
    print("     O_|    ")
    print("    /|\      ")
    print("    / \     ")

    return mistakes

In [91]:
def human(mask, guessed, **kwargs):
    return input().lower().strip()

In [92]:
word = random.choice(["ant", "pugger" , "tiger" , "pokemon" , "water" , "earth"])

hangman(word, human, 5, True)

『Hangman』，hints： _____ character :  5
5 turns left
df
Try Again！,Not valid character
4 turns left
g
3 turns left
bv
Try Again！,Not valid character
2 turns left
a
Guess the word:， _a___
2 turns left
z
1 turns left
h
『You loose』，The Ans : water
You loose, You let a kind man die
  --------  
     O_|    
    /|\      
    / \     


5

In [85]:
import random
print("This is a dice stimulator")
x = "y"

while x == "y":
    number = random.randint(1,6)

    if number == 1:
        print("----------")
        print("|        |")
        print("|    O   |")
        print("|        |")
        print("----------")
    if number == 2:
        print("----------")
        print("|        |")
        print("| O    O |")
        print("|        |")
        print("----------")
    if number == 3:
        print("----------")
        print("|    O   |")
        print("|    O   |")
        print("|    O   |")
        print("----------")
    if number == 4:
        print("----------")
        print("| O    O |")
        print("|        |")
        print("| O    O |")
        print("----------")
    if number == 5:
        print("----------")
        print("| O    O |")
        print("|    O   |")
        print("| O    O |")
        print("----------")
    if number == 6:
        print("----------")
        print("| O    O |")
        print("| O    O |")
        print("| O    O |")
        print("----------")
    x = input("Press y to roll again")


This is a dice stimulator
----------
| O    O |
| O    O |
| O    O |
----------
Press y to roll again


## 準備訓練和測試集

開始寫我們的AI猜字之前，要先準備訓練集和測試集。我們使用之前用過的Brown Corpus來訓練AI猜字演算法，為了刻意增加猜字的難度，AI在測試時的字和訓練的字會是不一樣的(把訓練集和測試集拆開)，如此才能訓練出更通用的模型。

我們使用 nltk.corpus.Brown 裡的 words() 方法，從中將非英文字母的文字去除，同時將每個字小寫化。之後，我們用 numpy.random.shuffle 來隨機刷取文集中的字，以分成訓練集和測試集。測試集裡有1000字，剩下的字都會到訓練集裡面。

In [ ]:
import nltk
nltk.download('brown')

In [87]:
from nltk.corpus import brown
import numpy as np

np.random.seed(12345)
word_set = []      # word_set 儲存Brown Corpus中所有獨特的字型
test_set = []      # test_set 儲存1000個字的測試集
training_set = []  # training_set 將剩下的字存到訓練集中
word_s = set([])

for word in brown.words():
    if word.isalpha():
        word = word.lower()
        if word not in word_s:
            word_s.add(word)

word_set = list(word_s)
np.random.shuffle(word_set)

test_set = word_set[:1000]
training_set = word_set[1000:]

print(len(word_set))
print(len(test_set))
print(len(training_set))

40234
1000
39234


In [89]:
hangman(np.random.choice(test_set), human, 5 , True)

『Hangman』，hints： _ _ _ _ _ _ _ _ _ character :  9
8 turns left
f
7 turns left
d
Guess the word:， _ _ _ _ _ d _ _ _
7 turns left
a
Guess the word:， _ a _ a _ d _ _ _
7 turns left
p
6 turns left
h
5 turns left
z
4 turns left
x
3 turns left
c
2 turns left
v
1 turns left
b
Guess the word:， _ a b a _ d _ _ _
1 turns left
q
『You loose』，The Ans : gabardine
You loose, You let a kind man die
  --------  
     O_|    
    /|\      
    / \     


8

## AI猜字方法：參考 rainrush
- 1.Random

In [98]:
def test_guesser(guesser, test=test_set):
    """
        這個方法是用來測試平均猜錯的次數。
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [99]:
import string

def random_guesser(mask, guessed, **kwargs):
    alphabets = []
    for letter in range(97,123):
        if chr(letter) not in guessed:
            alphabets.append(chr(letter))

    picked = np.random.choice(alphabets)
    return picked

# 若要看看機器是怎麼猜字的，可以把下面這句打開
#hangman(np.random.choice(test_set), random_guesser, 10, True)

result = test_guesser(random_guesser)

『Hangman』，hints： _______ character :  7
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
21 turns left
20 turns left
Guess the word:， m______
20 turns left
Guess the word:， mi_____
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， mirr_r_
15 turns left
14 turns left
13 turns left
12 turns left
11 turns left
10 turns left
Guess the word:， mirror_
10 turns left
9 turns left
8 turns left
7 turns left
6 turns left
5 turns left
Guess the word:， mirrors
『Hangman』，hints： __________ character :  10
26 turns left
25 turns left
24 turns left
Guess the word:， _____e____
24 turns left
23 turns left
22 turns left
21 turns left
20 turns left
Guess the word:， _____e__n_
20 turns left
Guess the word:， _____e__ng
20 turns left
Guess the word:， _____er_ng
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
Guess the word:， _w___er_ng
16 turns left
15 turns left
Guess the word:， _wi__ering
15 turns left
14 turns lef

26 turns left
25 turns left
24 turns left
Guess the word:， ___n___
24 turns left
23 turns left
Guess the word:， ___n_e_
23 turns left
22 turns left
Guess the word:， ___nge_
22 turns left
21 turns left
Guess the word:， t__nge_
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
14 turns left
Guess the word:， tw_nge_
14 turns left
13 turns left
12 turns left
11 turns left
10 turns left
9 turns left
8 turns left
Guess the word:， twinge_
8 turns left
Guess the word:， twinges
『Hangman』，hints： ______ character :  6
26 turns left
25 turns left
Guess the word:， __n___
25 turns left
Guess the word:， _in___
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， _in_a_
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
14 turns left
13 turns left
12 turns left
Guess the word:， tin_a_
12 turns left
11 turns left
10 turns left
9 turns left
Guess the word:， tinda_
9 turns left

18 turns left
17 turns left
16 turns left
15 turns left
14 turns left
Guess the word:， _nt_n_d
14 turns left
13 turns left
12 turns left
Guess the word:， int_n_d
12 turns left
11 turns left
Guess the word:， inton_d
11 turns left
10 turns left
9 turns left
8 turns left
7 turns left
6 turns left
Guess the word:， intoned
『Hangman』，hints： _________ character :  9
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
21 turns left
20 turns left
19 turns left
Guess the word:， u________
19 turns left
Guess the word:， us___ss__
19 turns left
18 turns left
Guess the word:， use_ess__
18 turns left
17 turns left
16 turns left
15 turns left
14 turns left
13 turns left
12 turns left
11 turns left
10 turns left
Guess the word:， use_ess_y
10 turns left
9 turns left
Guess the word:， uselessly
『Hangman』，hints： ______________ character :  14
26 turns left
Guess the word:， _e_e__________
26 turns left
25 turns left
Guess the word:， _e_er_________
25 turns left
24 turns left
23 turns left


18 turns left
17 turns left
16 turns left
Guess the word:， __s___us_____
16 turns left
Guess the word:， __s___us__n__
16 turns left
15 turns left
14 turns left
Guess the word:， __s___us_on__
14 turns left
13 turns left
Guess the word:， _isi__usion__
13 turns left
12 turns left
11 turns left
Guess the word:， _isillusion__
11 turns left
10 turns left
9 turns left
8 turns left
Guess the word:， _isillusione_
8 turns left
Guess the word:， disillusioned
『Hangman』，hints： ________ character :  8
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， _e_____e
22 turns left
Guess the word:， _e___c_e
22 turns left
Guess the word:， te_t_c_e
22 turns left
21 turns left
20 turns left
19 turns left
Guess the word:， tent_c_e
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， tentac_e
15 turns left
14 turns left
13 turns left
Guess the word:， tentacle
『Hangman』，hints： _________ character :  9
26 turns left
25 turns left
24 turns left
2

21 turns left
Guess the word:， _____m
21 turns left
20 turns left
Guess the word:， __a__m
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
Guess the word:， __a_om
16 turns left
Guess the word:， __alom
16 turns left
Guess the word:， _halom
16 turns left
15 turns left
14 turns left
13 turns left
12 turns left
11 turns left
Guess the word:， shalom
『Hangman』，hints： _____ character :  5
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， __i__
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
14 turns left
Guess the word:， __ic_
14 turns left
13 turns left
Guess the word:， __ice
13 turns left
12 turns left
11 turns left
10 turns left
9 turns left
8 turns left
7 turns left
Guess the word:， _rice
7 turns left
6 turns left
Guess the word:， price
『Hangman』，hints： _______ character :  7
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
21 turns left
Guess the 

...

Guess the word:， pun_
9 turns left
8 turns left
7 turns left
Guess the word:， puny
『Hangman』，hints： _______ character :  7
26 turns left
25 turns left
24 turns left
Guess the word:， ___n_n_
24 turns left
23 turns left
22 turns left
21 turns left
Guess the word:， _u_n_n_
21 turns left
20 turns left
19 turns left
18 turns left
Guess the word:， bu_n_n_
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， bu_n_ng
15 turns left
14 turns left
13 turns left
Guess the word:， bu_ning
13 turns left
12 turns left
Guess the word:， burning
『Hangman』，hints： _______ character :  7
26 turns left
Guess the word:， _____s_
26 turns left
25 turns left
Guess the word:， p____s_
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， p_b__s_
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
Guess the word:， p_bl_s_
18 turns left
17 turns left
16 turns left
15 turns left
14 turns left
Guess the word:， p_blis_
14 turns left
Guess the word:， p_blish
14 tu

23 turns left
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
Guess the word:， __m_c____
17 turns left
16 turns left
15 turns left
14 turns left
13 turns left
12 turns left
Guess the word:， __m_c__e_
12 turns left
11 turns left
Guess the word:， _om_c__e_
11 turns left
Guess the word:， _om_c_le_
11 turns left
10 turns left
9 turns left
Guess the word:， dom_c_led
9 turns left
8 turns left
7 turns left
Guess the word:， domiciled
『Hangman』，hints： _______ character :  7
26 turns left
25 turns left
Guess the word:， ___u___
25 turns left
24 turns left
23 turns left
22 turns left
21 turns left
20 turns left
19 turns left
Guess the word:， ___us__
19 turns left
18 turns left
17 turns left
16 turns left
Guess the word:， e__use_
16 turns left
15 turns left
Guess the word:， e_cuse_
15 turns left
14 turns left
13 turns left
Guess the word:， e_cused
13 turns left
12 turns left
11 turns left
10 turns left
9 turns left
8 turns left
7 turns left
6 turns left
Guess the

16 turns left
Guess the word:， h___m_____
16 turns left
15 turns left
14 turns left
Guess the word:， h___m___e_
14 turns left
13 turns left
12 turns left
Guess the word:， h___m_i_e_
12 turns left
Guess the word:， h__dm_ide_
12 turns left
11 turns left
10 turns left
Guess the word:， ha_dmaide_
10 turns left
9 turns left
8 turns left
7 turns left
Guess the word:， handmaiden
『Hangman』，hints： _______ character :  7
26 turns left
25 turns left
24 turns left
Guess the word:， ___d___
24 turns left
23 turns left
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
Guess the word:， __od___
17 turns left
16 turns left
15 turns left
14 turns left
Guess the word:， i_odi__
14 turns left
13 turns left
Guess the word:， isodi__
13 turns left
12 turns left
Guess the word:， isodin_
12 turns left
11 turns left
10 turns left
9 turns left
Guess the word:， isodine
『Hangman』，hints： _________ character :  9
26 turns left
Guess the word:， ______i__
26 turns left
25 turns left
24 

17 turns left
16 turns left
15 turns left
14 turns left
13 turns left
12 turns left
11 turns left
10 turns left
Guess the word:， deepest
『Hangman』，hints： _______ character :  7
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
21 turns left
Guess the word:， __o____
21 turns left
20 turns left
Guess the word:， _po____
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
Guess the word:， _pou___
16 turns left
15 turns left
14 turns left
13 turns left
12 turns left
11 turns left
10 turns left
9 turns left
8 turns left
7 turns left
Guess the word:， _pou_e_
7 turns left
6 turns left
5 turns left
Guess the word:， spouses
『Hangman』，hints： ____________ character :  12
26 turns left
Guess the word:， __________i_
26 turns left
Guess the word:， _a____a___i_
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， _a___ma___i_
22 turns left
Guess the word:， _al__ma___i_
22 turns left
21 turns left
20 turns left
19 turns left
Guess 

24 turns left
Guess the word:， __g_____
24 turns left
Guess the word:， __gh____
24 turns left
23 turns left
Guess the word:， __ght___
23 turns left
22 turns left
Guess the word:， __ghtf__
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
Guess the word:， r_ghtf__
18 turns left
17 turns left
Guess the word:， r_ghtfu_
17 turns left
16 turns left
15 turns left
14 turns left
Guess the word:， r_ghtful
14 turns left
13 turns left
12 turns left
11 turns left
10 turns left
9 turns left
8 turns left
Guess the word:， rightful
『Hangman』，hints： _________ character :  9
26 turns left
25 turns left
24 turns left
23 turns left
Guess the word:， ___u_u___
23 turns left
Guess the word:， s__u_us__
23 turns left
22 turns left
Guess the word:， s__uous__
22 turns left
21 turns left
Guess the word:， s__uousl_
21 turns left
20 turns left
19 turns left
Guess the word:， si_uousl_
19 turns left
Guess the word:， si_uously
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
14

In [100]:
print("Average number of incorrect guesses: ", result)

Average number of incorrect guesses:  16.71


- 第二種猜法：Unigram Guesser

我們可以嘗試用Unigram模型來訓練。我們需要知道每個字母的出現頻率，接著照出現頻率的高低來進行猜字。每當猜完一個字之後就應該把猜過的字去掉。

In [101]:
from collections import Counter 
unigram_counts = Counter() # 儲存了整個訓練及中每個字母的出現次數

for word in training_set:
    for letter in word:
        unigram_counts[letter] += 1

print(unigram_counts)


def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):
    unigram_keys = []

    # 照出現頻率將字母排序
    for i in range(len(unigram_counts)):
        unigram_keys.append(unigram_counts.most_common()[i][0])

    # 將猜過的字去除
    for letter in guessed:
        if letter in unigram_keys:
            unigram_keys.remove(letter)

    return unigram_keys[0]

#hangman(np.random.choice(test_set), unigram_guesser, 10, True)
result = test_guesser(unigram_guesser)

Counter({'e': 35177, 'i': 26054, 'a': 24439, 's': 23689, 'n': 22616, 'r': 22027, 't': 20677, 'o': 19171, 'l': 16651, 'c': 12604, 'd': 12261, 'u': 10154, 'm': 8677, 'g': 8628, 'p': 8557, 'h': 7349, 'b': 5721, 'y': 5367, 'f': 4210, 'v': 3447, 'k': 2984, 'w': 2790, 'z': 1037, 'x': 840, 'j': 641, 'q': 525})
『Hangman』，hints： _______ character :  7
26 turns left
25 turns left
Guess the word:， _i_____
25 turns left
24 turns left
Guess the word:， _i____s
24 turns left
23 turns left
Guess the word:， _irr_rs
23 turns left
22 turns left
Guess the word:， _irrors
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
Guess the word:， mirrors
『Hangman』，hints： __________ character :  10
26 turns left
Guess the word:， _____e____
26 turns left
Guess the word:， __i__e_i__
26 turns left
25 turns left
24 turns left
Guess the word:， __i__e_in_
24 turns left
Guess the word:， __i__erin_
24 turns left
Guess the word:， t_itterin_
24 turns left
23 turns left
22 turns left
21 turns left
20 turns l

『Hangman』，hints： ___ character :  3
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， _n_
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
14 turns left
Guess the word:， gn_
14 turns left
Guess the word:， gnp
『Hangman』，hints： __________ character :  10
26 turns left
Guess the word:， _______e__
26 turns left
25 turns left
24 turns left
Guess the word:， _______ess
24 turns left
Guess the word:， ____n__ess
24 turns left
Guess the word:， _r__n__ess
24 turns left
23 turns left
Guess the word:， _ro_n__ess
23 turns left
Guess the word:， _ro_n_less
23 turns left
22 turns left
Guess the word:， _ro_ndless
22 turns left
Guess the word:， _roundless
22 turns left
21 turns left
Guess the word:， groundless
『Hangman』，hints： _____ character :  5
26 turns left
25 turns left
Guess the word:， ___i_
25 turns left
Guess the word:， __ai_
25 turns left
24 turns left
23 turns left
Guess the word:， __air
23 turns

25 turns left
24 turns left
Guess the word:， __ris____er
24 turns left
Guess the word:， __rist___er
24 turns left
Guess the word:， __risto__er
24 turns left
23 turns left
Guess the word:， c_risto__er
23 turns left
22 turns left
21 turns left
20 turns left
19 turns left
Guess the word:， c_ristop_er
19 turns left
Guess the word:， christopher
『Hangman』，hints： ____________ character :  12
26 turns left
Guess the word:， ____e_______
26 turns left
Guess the word:， ____e____i__
26 turns left
Guess the word:， ____e__a_i__
26 turns left
Guess the word:， ___se__a_i__
26 turns left
Guess the word:， __nse__a_i_n
26 turns left
Guess the word:， __nse_ra_i_n
26 turns left
Guess the word:， __nse_rati_n
26 turns left
Guess the word:， _onse_ration
26 turns left
25 turns left
Guess the word:， consecration
『Hangman』，hints： _______ character :  7
26 turns left
Guess the word:， ______e
26 turns left
Guess the word:， i_____e
26 turns left
Guess the word:， i___a_e
26 turns left
25 turns left
Guess the word:， 

19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
14 turns left
13 turns left
Guess the word:， hea_e
13 turns left
12 turns left
11 turns left
10 turns left
Guess the word:， heave
『Hangman』，hints： _______ character :  7
26 turns left
Guess the word:， ___e_e_
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， ___ere_
22 turns left
21 turns left
Guess the word:， _o_ere_
21 turns left
20 turns left
19 turns left
Guess the word:， _o_ered
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， ho_ered
15 turns left
14 turns left
13 turns left
12 turns left
Guess the word:， hovered
『Hangman』，hints： _________ character :  9
26 turns left
Guess the word:， _______ee
26 turns left
Guess the word:， ____i__ee
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， t___i__ee
22 turns left
Guess the word:， to__i__ee
22 turns left
21 turns left
20 turns left
19 turns left
18 turns lef

10 turns left
Guess the word:， sawdust
『Hangman』，hints： _____ character :  5
26 turns left
Guess the word:， ____e
26 turns left
25 turns left
24 turns left
23 turns left
Guess the word:， __n_e
23 turns left
22 turns left
21 turns left
Guess the word:， o_n_e
21 turns left
20 turns left
Guess the word:， o_nce
20 turns left
19 turns left
Guess the word:， ounce
『Hangman』，hints： ________ character :  8
26 turns left
Guess the word:， ___e__e_
26 turns left
Guess the word:， i__e__e_
26 turns left
25 turns left
24 turns left
Guess the word:， in_e__e_
24 turns left
23 turns left
Guess the word:， in_e_te_
23 turns left
22 turns left
21 turns left
20 turns left
Guess the word:， inde_ted
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， indebted
『Hangman』，hints： _____ character :  5
26 turns left
25 turns left
24 turns left
Guess the word:， __a__
24 turns left
Guess the word:， __a_s
24 turns left
23 turns left
22 turns left
21 turns left
20 turns 

...

Guess the word:， ___e________e
26 turns left
Guess the word:， i__e_____i__e
26 turns left
Guess the word:， i__e__a__ia_e
26 turns left
25 turns left
Guess the word:， in_e__a__ia_e
25 turns left
Guess the word:， in_er_arria_e
25 turns left
Guess the word:， inter_arria_e
25 turns left
24 turns left
23 turns left
22 turns left
21 turns left
20 turns left
Guess the word:， intermarria_e
20 turns left
Guess the word:， intermarriage
『Hangman』，hints： _____________ character :  13
26 turns left
Guess the word:， _e______e____
26 turns left
Guess the word:， _e_i____e_i__
26 turns left
25 turns left
Guess the word:， _e_is___e_i__
25 turns left
Guess the word:， _e_is___e_in_
25 turns left
Guess the word:， re_is___erin_
25 turns left
24 turns left
Guess the word:， re_is_o_erin_
24 turns left
23 turns left
Guess the word:， re_isco_erin_
23 turns left
Guess the word:， redisco_erin_
23 turns left
22 turns left
21 turns left
Guess the word:， redisco_ering
21 turns left
20 turns left
19 turns left
18 tur

11 turns left
10 turns left
9 turns left
Guess the word:， excitement
『Hangman』，hints： _______ character :  7
26 turns left
25 turns left
24 turns left
23 turns left
Guess the word:， ___s__s
23 turns left
Guess the word:， ___s_ns
23 turns left
22 turns left
21 turns left
Guess the word:， ___sons
21 turns left
20 turns left
19 turns left
Guess the word:， __dsons
19 turns left
Guess the word:， _udsons
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
14 turns left
13 turns left
12 turns left
11 turns left
10 turns left
9 turns left
8 turns left
7 turns left
Guess the word:， judsons
『Hangman』，hints： ______ character :  6
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
Guess the word:， ____l_
18 turns left
17 turns left
16 turns left
Guess the word:， _u__l_
16 turns left
15 turns left
14 turns left
13 turns left
12 turns left
Guess the word:， bubbl_
12 turns left
Guess the word:， bubbly
『Hangma

Guess the word:， e_______e
26 turns left
25 turns left
Guess the word:， e____a__e
25 turns left
24 turns left
Guess the word:， en___a__e
24 turns left
Guess the word:， en__ra__e
24 turns left
23 turns left
22 turns left
Guess the word:， en__ra_le
22 turns left
21 turns left
Guess the word:， end_ra_le
21 turns left
Guess the word:， endura_le
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
Guess the word:， endurable
『Hangman』，hints： _____ character :  5
26 turns left
25 turns left
24 turns left
Guess the word:， _a___
24 turns left
23 turns left
22 turns left
21 turns left
Guess the word:， _at__
21 turns left
20 turns left
19 turns left
Guess the word:， _atc_
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， patc_
15 turns left
Guess the word:， patch
『Hangman』，hints： ____________ character :  12
26 turns left
Guess the word:， ___e___e____
26 turns left
25 turns left
Guess the word:， ___e___e__a_
25 turns left
24 turns left
Guess t

23 turns left
Guess the word:， _ar_e
23 turns left
22 turns left
21 turns left
Guess the word:， lar_e
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
Guess the word:， large
『Hangman』，hints： _____ character :  5
26 turns left
25 turns left
24 turns left
Guess the word:， _a___
24 turns left
23 turns left
22 turns left
Guess the word:， _ar__
22 turns left
21 turns left
Guess the word:， _ar_o
21 turns left
20 turns left
19 turns left
18 turns left
Guess the word:， _aruo
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， haruo
『Hangman』，hints： _______ character :  7
26 turns left
Guess the word:， _ee_e__
26 turns left
25 turns left
24 turns left
Guess the word:， _ee_es_
24 turns left
23 turns left
22 turns left
Guess the word:， _ee_est
22 turns left
21 turns left
20 turns left
19 turns left
Guess the word:， dee_est
19 turns left
18 turns left
17 turns left
16 turns left
Guess the word:， deepest
『Hangman』，hints： _______ character :  7
26 turns left

Guess the word:， _nset
23 turns left
Guess the word:， onset
『Hangman』，hints： ________ character :  8
26 turns left
Guess the word:， ______e_
26 turns left
25 turns left
Guess the word:， _a____e_
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， tatt__e_
22 turns left
Guess the word:， tattooe_
22 turns left
21 turns left
20 turns left
Guess the word:， tattooed
『Hangman』，hints： __________ character :  10
26 turns left
Guess the word:， _e_e______
26 turns left
Guess the word:， _e_e_i____
26 turns left
25 turns left
Guess the word:， _e_e_is___
25 turns left
24 turns left
Guess the word:， _e_eris___
24 turns left
23 turns left
22 turns left
Guess the word:， _e_eris_l_
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
Guess the word:， _e_erishl_
16 turns left
15 turns left
Guess the word:， _e_erishly
15 turns left
Guess the word:， fe_erishly
15 turns left
Guess the word:， feverishly
『Hangman』，hints： _______ character :  7

In [102]:
print("平均猜錯次數：", result)

平均猜錯次數： 10.171


- 第三種猜法：根據文字長度猜字
從和昨天同一篇文章中我們看到，不同的文字長度，每個字母出現的頻率不盡相同，例如，短的字比較不會出現前綴或後綴。在這裡，我們針對不同的文字長度設計不一樣的猜字順序

In [103]:
from collections import defaultdict

# unigram_counts_by_length 將文字長度和字母頻率map在一起
unigram_counts_by_length = defaultdict(Counter)

# 幫每一種文字長度寫不同的Unigram Model
for word in training_set:    
    this_count = Counter()
    for letter in word:
        this_count[letter] += 1
        unigram_counts_by_length[len(word)] += this_count
        this_count = Counter()

def exclude_guessed_letters(length_model, guessed):
    unigram_keys_by_length = []
    # 照出現頻率將字母排序
    for i in range(len(unigram_counts_by_length[length_model])):
        unigram_keys_by_length.append(unigram_counts_by_length[length_model].most_common()[i][0])
    
    # 將猜過的字去除
    for letter in guessed:
        if letter in unigram_keys_by_length:
            unigram_keys_by_length.remove(letter)
    return unigram_keys_by_length


lengths = sorted(unigram_counts_by_length.keys())
max_length = lengths[-1] + 1

print(unigram_counts_by_length)

def unigram_length_guesser(mask, guessed, counts=unigram_counts_by_length):
    
    length_model = len(mask)
    # 若要猜的文字長度不在unigram model時，我們將一長度來猜。
    while length_model not in lengths:
        length_model -= 1
    
    unigram_keys_by_length = exclude_guessed_letters(length_model, guessed)
    
    # 若這個文字長度沒有猜字選項了，從附近的文字長度找
    while len(unigram_keys_by_length) == 0:
        if length_model < 20:
            length_model += 1
        else:
            length_model -= 1
        unigram_keys_by_length = exclude_guessed_letters(length_model, guessed)
    
    return unigram_keys_by_length[0]
#hangman(np.random.choice(test_set), unigram_length_guesser, 10, True)
result = test_guesser(unigram_length_guesser)

defaultdict(<class 'collections.Counter'>, {12: Counter({'i': 1902, 'e': 1891, 'n': 1540, 't': 1474, 'a': 1434, 's': 1308, 'r': 1265, 'o': 1230, 'l': 969, 'c': 900, 'u': 588, 'p': 557, 'm': 522, 'd': 511, 'g': 391, 'h': 387, 'y': 296, 'b': 257, 'f': 199, 'v': 194, 'z': 71, 'k': 53, 'w': 51, 'x': 40, 'q': 36, 'j': 18}), 7: Counter({'e': 5298, 'a': 3556, 's': 3448, 'r': 3390, 'i': 3327, 'n': 3025, 't': 2650, 'o': 2494, 'l': 2476, 'd': 2048, 'c': 1677, 'g': 1493, 'u': 1483, 'p': 1236, 'm': 1232, 'h': 1102, 'b': 931, 'y': 749, 'f': 667, 'k': 585, 'w': 521, 'v': 461, 'z': 151, 'x': 111, 'j': 97, 'q': 81}), 11: Counter({'e': 2950, 'i': 2745, 'n': 2256, 't': 2158, 'a': 2010, 's': 1999, 'r': 1904, 'o': 1699, 'l': 1323, 'c': 1230, 'u': 854, 'd': 803, 'p': 776, 'm': 739, 'g': 632, 'h': 518, 'b': 420, 'y': 416, 'f': 289, 'v': 287, 'k': 133, 'w': 120, 'x': 73, 'z': 71, 'q': 46, 'j': 26}), 3: Counter({'a': 237, 'e': 198, 'o': 177, 'i': 126, 'n': 118, 't': 115, 's': 110, 'p': 103, 'm': 101, 'u': 99,

26 turns left
Guess the word:， __ees
26 turns left
25 turns left
24 turns left
23 turns left
Guess the word:， _lees
23 turns left
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
14 turns left
13 turns left
12 turns left
11 turns left
10 turns left
Guess the word:， flees
『Hangman』，hints： ________ character :  8
26 turns left
25 turns left
24 turns left
23 turns left
Guess the word:， s_____ss
23 turns left
22 turns left
Guess the word:， s___r_ss
22 turns left
21 turns left
Guess the word:， s___ross
21 turns left
20 turns left
19 turns left
18 turns left
Guess the word:， su__ross
18 turns left
Guess the word:， su_gross
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， subgross
『Hangman』，hints： _____ character :  5
26 turns left
Guess the word:， ___e_
26 turns left
Guess the word:， ___es
26 turns left
25 turns left
24 turns left
Guess the word:， _o_es
24 turns left
23 turns left
22 turns left
21 turns le

Guess the word:， in_la_e
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
Guess the word:， in_lame
17 turns left
16 turns left
15 turns left
14 turns left
Guess the word:， inflame
『Hangman』，hints： _______ character :  7
26 turns left
25 turns left
Guess the word:， _a___a_
25 turns left
24 turns left
Guess the word:， _a_r_a_
24 turns left
Guess the word:， _a_ria_
24 turns left
Guess the word:， _a_rian
24 turns left
23 turns left
22 turns left
21 turns left
Guess the word:， _adrian
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
Guess the word:， hadrian
『Hangman』，hints： ________ character :  8
26 turns left
Guess the word:， _e_e__e_
26 turns left
Guess the word:， _e_ei_e_
26 turns left
25 turns left
Guess the word:， _e_ei_es
25 turns left
24 turns left
23 turns left
22 turns left
21 turns left
20 turns left
Guess the word:， de_ei_es
20 turns left
Guess the word:， decei_es
20 turns left
19 turns left
18 turns left
17 t

Guess the word:， si_
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
14 turns left
13 turns left
12 turns left
11 turns left
10 turns left
9 turns left
8 turns left
7 turns left
6 turns left
5 turns left
Guess the word:， six
『Hangman』，hints： ___________ character :  11
26 turns left
Guess the word:， _e__e_____e
26 turns left
25 turns left
Guess the word:， _e__e___n_e
25 turns left
Guess the word:， _e__e_t_n_e
25 turns left
Guess the word:， _e__e_tan_e
25 turns left
24 turns left
Guess the word:， re__e_tan_e
24 turns left
23 turns left
Guess the word:， re_le_tan_e
23 turns left
Guess the word:， re_lectance
23 turns left
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， reflectance
『Hangman』，hints： ____________ character :  12
26 turns left
Guess the word:， i___________
26 turns left
Guess the word:， i__________e
26 turns left
25 turns left
24 turns left
Guess 

23 turns left
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， transferral
『Hangman』，hints： _____________ character :  13
26 turns left
Guess the word:， ______i_i____
26 turns left
Guess the word:， ___e__i_i____
26 turns left
25 turns left
24 turns left
Guess the word:， ___e__i_ia___
24 turns left
23 turns left
Guess the word:， s__e__i_ia___
23 turns left
Guess the word:， s__er_i_ia___
23 turns left
Guess the word:， s__er_icia___
23 turns left
Guess the word:， s__er_iciall_
23 turns left
Guess the word:， s_per_iciall_
23 turns left
22 turns left
Guess the word:， super_iciall_
22 turns left
21 turns left
20 turns left
19 turns left
Guess the word:， super_icially
19 turns left
18 turns left
17 turns left
Guess the word:， superficially
『Hangman』，hints： ________ character :  8
26 turns left
25 turns left
Guess the word:， i__i____
25 turns left
24 turns left
Guess the word:， i_si___s
24 turns left
Guess the word

...

Guess the word:， __e_a__en_
24 turns left
Guess the word:， __e_a__ent
24 turns left
Guess the word:， _re_a__ent
24 turns left
23 turns left
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
Guess the word:， prepa__ent
18 turns left
Guess the word:， prepa_ment
18 turns left
17 turns left
16 turns left
Guess the word:， prepayment
『Hangman』，hints： _______ character :  7
26 turns left
Guess the word:， _____e_
26 turns left
Guess the word:， __a__e_
26 turns left
Guess the word:， s_a__es
26 turns left
25 turns left
24 turns left
Guess the word:， s_an_es
24 turns left
Guess the word:， stan_es
24 turns left
23 turns left
22 turns left
21 turns left
Guess the word:， stances
『Hangman』，hints： ________ character :  8
26 turns left
Guess the word:， ______e_
26 turns left
25 turns left
24 turns left
23 turns left
Guess the word:， ___n__e_
23 turns left
Guess the word:， __rn__er
23 turns left
Guess the word:， t_rn__er
23 turns left
Guess the word:， t_rno_er
23 turns left
22 turns 

Guess the word:， _olonialis_
23 turns left
Guess the word:， colonialis_
23 turns left
22 turns left
21 turns left
20 turns left
Guess the word:， colonialism
『Hangman』，hints： ____ character :  4
26 turns left
25 turns left
24 turns left
23 turns left
Guess the word:， _o__
23 turns left
22 turns left
21 turns left
Guess the word:， _o_r
21 turns left
20 turns left
19 turns left
Guess the word:， do_r
19 turns left
Guess the word:， dour
『Hangman』，hints： ________ character :  8
26 turns left
Guess the word:， ___e____
26 turns left
Guess the word:， ___e_i__
26 turns left
25 turns left
24 turns left
Guess the word:， ___e_in_
24 turns left
23 turns left
22 turns left
Guess the word:， _o_e_in_
22 turns left
Guess the word:， _o_elin_
22 turns left
Guess the word:， do_elin_
22 turns left
21 turns left
20 turns left
Guess the word:， do_eling
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
14 turns left
13 turns left
12 turns left
Guess the word:， doweling
『Hangma

20 turns left
19 turns left
18 turns left
17 turns left
Guess the word:， sem_er
17 turns left
16 turns left
15 turns left
Guess the word:， semper
『Hangman』，hints： _______ character :  7
26 turns left
Guess the word:， ____e__
26 turns left
25 turns left
24 turns left
Guess the word:， _r__e__
24 turns left
23 turns left
Guess the word:， _r__en_
23 turns left
Guess the word:， _r__ent
23 turns left
22 turns left
21 turns left
Guess the word:， _r_dent
21 turns left
20 turns left
19 turns left
Guess the word:， _rudent
19 turns left
Guess the word:， prudent
『Hangman』，hints： ______ character :  6
26 turns left
25 turns left
Guess the word:， ____a_
25 turns left
Guess the word:， s___a_
25 turns left
24 turns left
23 turns left
Guess the word:， s_n_an
23 turns left
22 turns left
21 turns left
20 turns left
19 turns left
Guess the word:， sun_an
19 turns left
18 turns left
Guess the word:， sunman
『Hangman』，hints： _________ character :  9
26 turns left
Guess the word:， ___e_____
26 turns left
Guess

25 turns left
24 turns left
23 turns left
22 turns left
21 turns left
Guess the word:， _i__
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， hi_h
15 turns left
14 turns left
13 turns left
12 turns left
11 turns left
Guess the word:， high
『Hangman』，hints： _______ character :  7
26 turns left
Guess the word:， _____e_
26 turns left
Guess the word:， __a__e_
26 turns left
25 turns left
Guess the word:， _ra__e_
25 turns left
24 turns left
23 turns left
Guess the word:， _ra__et
23 turns left
22 turns left
21 turns left
20 turns left
Guess the word:， _rac_et
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， brac_et
15 turns left
14 turns left
13 turns left
Guess the word:， bracket
『Hangman』，hints： ______ character :  6
26 turns left
Guess the word:， __e___
26 turns left
25 turns left
24 turns left
Guess the word:， _re___
24 turns left
23 turns left
22 turns left
21 turns left
20

21 turns left
Guess the word:， unmista_a_l_
21 turns left
20 turns left
19 turns left
18 turns left
Guess the word:， unmista_a_ly
18 turns left
Guess the word:， unmista_ably
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， unmistakably
『Hangman』，hints： _______ character :  7
26 turns left
Guess the word:， _e___e_
26 turns left
Guess the word:， _ea__e_
26 turns left
Guess the word:， _ea__es
26 turns left
Guess the word:， rea__es
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
21 turns left
20 turns left
Guess the word:， reac_es
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
Guess the word:， reaches
『Hangman』，hints： _____________ character :  13
26 turns left
Guess the word:， ___________i_
26 turns left
25 turns left
Guess the word:， ________n__i_
25 turns left
Guess the word:， ____t___n__i_
25 turns left
Guess the word:， ____t__an__i_
25 turns left
Guess the word:， __o_t__an__i_
25 turns left
Guess the word:， s_o_ts_an

In [104]:
print("平均猜錯次數：", result)

平均猜錯次數： 10.136


- 強化版本

除了考慮字母出現的機率和在各種長度中各個字母出現的機率，今天我們也把字母的排列順序列入考量。例如，我們看到一個字 m _ s s，我們知道他有很高的機率會是母音，因此，a, e, i, o, u就是我們下一個猜測的目標。

在這裡，我們開發一個Bigram的語言模型。在開始訓練這個模型之前，有幾件事需要注意：(1) 要記得加上開頭標籤 $；(2) 要記得針對Bigram以上的模型設計對應的平滑方法。在這裡我們決定使用 linear interpolation （線性插值法）來smooth高元和低元模型，而插值法中要用到的 \lambda 值我們設為0.75（建議在0.7~0.8之間）。

我們猜格子的順序會從最左邊開始猜，因此最開始會在p($)的條件下猜字母。猜的過程中可能第一個字還沒猜到，後面有些就先被猜到了，等我們終於猜到第一個字母後，我們就找下一個「最左邊還沒有被猜到的字」。

In [106]:
from operator import itemgetter
bigram_counts = defaultdict(Counter) # 這個dict的key是各個字母，value是可能接在這個字母後面之字母的Counter
bigram_inter = defaultdict(list) # 根據interpolation後的機率，將接續在一個字母後面的可能字母照出現機率排序

# 開始儲存Bigram model
for word in training_set:
    this_count = Counter()
    word = '$' + word # 字首加上開頭標籤
    for i, letter in enumerate(word):
        if i+1 != len(word):
            this_count[word[i+1]] += 1
            bigram_counts[letter] += this_count
            this_count = Counter()

set_lambda = 0.75 # 設定interpolation的lambda

# Bigram Interpolated Model: p = lambda*p(w_i|w_(i-1)) + (1-lambda)*p(w_i)
# p(w_i|w_(i-1)) = count(w_i and w_(i-1)) / count(w_(i-1))
# p(w_i) = count(w_i) / sigma(count(w_i))
for key in bigram_counts.keys():
    sigma_count_wi = sum(unigram_counts.values()) # sigma(count(w_i))
    count_wi1 = sum(bigram_counts[key].values()) # count(w_(i-1))

    # 計算p(w_i|w_(i-1))和p(w_i)
    prob_of_letter = {}
    for letter in range(97,123):
        p_wi_wi1 = bigram_counts[key][chr(letter)] / count_wi1 # p(w_i|w_(i-1))
        p_wi = unigram_counts[chr(letter)] / sigma_count_wi # p(w_i)
        prob_of_letter[chr(letter)] = set_lambda*p_wi_wi1 + (1-set_lambda)*p_wi

    # 將接續在一個字母後面的可能字母照出現機率排序
    this_list = []
    for i in range(26):
        this_list.append(sorted(prob_of_letter.items(), key=itemgetter(1), reverse=True)[i][0])
    bigram_inter[key] = this_list
    

def bigram_guesser(mask, guessed, counts=bigram_counts): # add extra default arguments if needed
    mask = ['$'] + mask
    w_i_1 = "" # 找到最左邊非 '_' 的字母 -> w_(i-1)
    # 找w_i_1
    for i in range(len(mask)):
        if mask[i] == '_':
            w_i_1 = mask[i-1]
            break
    copy_bi_model = bigram_inter[w_i_1].copy()
    
    # 將已經猜過的字母去除
    for letter in guessed:
        if letter in copy_bi_model:
            copy_bi_model.remove(letter)

    return copy_bi_model[0]


#hangman(np.random.choice(test_set), bigram_guesser, 26, True)

result = test_guesser(bigram_guesser)

『Hangman』，hints： _______ character :  7
26 turns left
Guess the word:， ______s
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， __rr_rs
22 turns left
21 turns left
20 turns left
Guess the word:， _irr_rs
20 turns left
19 turns left
Guess the word:， mirr_rs
19 turns left
Guess the word:， mirrors
『Hangman』，hints： __________ character :  10
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， _____e____
22 turns left
Guess the word:， _____er___
22 turns left
21 turns left
Guess the word:， t__tter___
21 turns left
Guess the word:， t_itteri__
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
Guess the word:， t_itterin_
16 turns left
15 turns left
Guess the word:， t_ittering
15 turns left
14 turns left
13 turns left
Guess the word:， twittering
『Hangman』，hints： _________ character :  9
26 turns left
Guess the word:， s________
26 turns left
25 turns left
24 turns left
23 turns left
Guess th

Guess the word:， ___e_____e__e
22 turns left
Guess the word:， __re_____e__e
22 turns left
Guess the word:， __re_____ed_e
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
Guess the word:， __re____led_e
18 turns left
Guess the word:， f_re____led_e
18 turns left
Guess the word:， fore__o_led_e
18 turns left
Guess the word:， fore_no_led_e
18 turns left
Guess the word:， fore_no_ledge
18 turns left
17 turns left
16 turns left
15 turns left
14 turns left
13 turns left
Guess the word:， fore_nowledge
13 turns left
Guess the word:， foreknowledge
『Hangman』，hints： _________ character :  9
26 turns left
Guess the word:， _____s___
26 turns left
25 turns left
24 turns left
23 turns left
Guess the word:， ___ees__e
23 turns left
Guess the word:， __rees__e
23 turns left
22 turns left
Guess the word:， t_rees__e
22 turns left
21 turns left
Guess the word:， t_reeso_e
21 turns left
Guess the word:， threeso_e
21 turns left
20 turns left
19 turns left
18 turns left
Guess the word:， threeso

7 turns left
Guess the word:， quack
『Hangman』，hints： ______________ character :  14
26 turns left
Guess the word:， _______s_____s
26 turns left
25 turns left
Guess the word:， a______s_____s
25 turns left
Guess the word:， a____t_s____ts
25 turns left
Guess the word:， a____t_s___nts
25 turns left
Guess the word:， a___rt_s___nts
25 turns left
24 turns left
Guess the word:， a___rtis___nts
24 turns left
Guess the word:， ad__rtis___nts
24 turns left
Guess the word:， ad_ertise_ents
24 turns left
23 turns left
22 turns left
21 turns left
20 turns left
Guess the word:， ad_ertisements
20 turns left
19 turns left
18 turns left
Guess the word:， advertisements
『Hangman』，hints： _________ character :  9
26 turns left
Guess the word:， ____s____
26 turns left
25 turns left
24 turns left
23 turns left
Guess the word:， ___es____
23 turns left
22 turns left
21 turns left
Guess the word:， ___est___
21 turns left
Guess the word:， i__esti__
21 turns left
Guess the word:， in_estin_
21 turns left
Guess the wor

26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， _e__e
22 turns left
Guess the word:， re__e
22 turns left
21 turns left
Guess the word:， re_ne
21 turns left
20 turns left
19 turns left
Guess the word:， reine
『Hangman』，hints： _____ character :  5
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， ____e
22 turns left
Guess the word:， r___e
22 turns left
21 turns left
Guess the word:， ro__e
21 turns left
20 turns left
Guess the word:， rou_e
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， rouge
『Hangman』，hints： ______ character :  6
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
21 turns left
Guess the word:， __r___
21 turns left
20 turns left
19 turns left
Guess the word:， _ir___
19 turns left
18 turns left
Guess the word:， _irm__
18 turns left
Guess the word:， _irml_
18 turns left
Guess the word:， firml_
18 turns left
Guess the word:， firml

25 turns left
Guess the word:， s__t_i_hts
25 turns left
24 turns left
Guess the word:， s_ot_i_hts
24 turns left
23 turns left
22 turns left
Guess the word:， spot_i_hts
22 turns left
21 turns left
Guess the word:， spotli_hts
21 turns left
20 turns left
19 turns left
18 turns left
Guess the word:， spotlights
『Hangman』，hints： ____________ character :  12
26 turns left
Guess the word:， ___ss_______
26 turns left
25 turns left
Guess the word:， ___ss___a___
25 turns left
24 turns left
Guess the word:， ___ss__ea___
24 turns left
23 turns left
22 turns left
Guess the word:， ___ss__ea__t
22 turns left
Guess the word:， __iss__ea__t
22 turns left
Guess the word:， b_iss__ea__t
22 turns left
Guess the word:， b_iss__ea_lt
22 turns left
Guess the word:， boisso_ea_lt
22 turns left
Guess the word:， boissonea_lt
22 turns left
21 turns left
20 turns left
Guess the word:， boissoneault
『Hangman』，hints： _______ character :  7
26 turns left
25 turns left
24 turns left
Guess the word:， ____a__
24 turns left
2

...

Guess the word:， ec__pse
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， ec_ipse
22 turns left
21 turns left
20 turns left
Guess the word:， eclipse
『Hangman』，hints： _____ character :  5
26 turns left
Guess the word:， ____s
26 turns left
25 turns left
Guess the word:， _a__s
25 turns left
24 turns left
Guess the word:， _a_es
24 turns left
23 turns left
22 turns left
Guess the word:， _ates
22 turns left
21 turns left
Guess the word:， bates
『Hangman』，hints： _____ character :  5
26 turns left
25 turns left
24 turns left
Guess the word:， _a___
24 turns left
23 turns left
Guess the word:， _a_e_
23 turns left
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
Guess the word:， _a_el
17 turns left
16 turns left
15 turns left
Guess the word:， na_el
15 turns left
14 turns left
13 turns left
12 turns left
11 turns left
Guess the word:， navel
『Hangman』，hints： ____ character :  4
26 turns left
Guess the word:， s___
26 turns left
25 turns left


Guess the word:， s__ste_i_i_atio_
25 turns left
24 turns left
Guess the word:， su_ste_i_i_atio_
24 turns left
Guess the word:， su_ste_i_i_ation
24 turns left
Guess the word:， su_steri_i_ation
24 turns left
Guess the word:， su_sterili_ation
24 turns left
23 turns left
22 turns left
21 turns left
Guess the word:， substerili_ation
21 turns left
20 turns left
19 turns left
18 turns left
Guess the word:， substerilization
『Hangman』，hints： ___________ character :  11
26 turns left
25 turns left
24 turns left
23 turns left
Guess the word:， p__________
23 turns left
22 turns left
Guess the word:， pr_________
22 turns left
Guess the word:， pr______i__
22 turns left
Guess the word:， pro_o___i__
22 turns left
Guess the word:， pro_o_n_i__
22 turns left
Guess the word:， pro_oun_i__
22 turns left
21 turns left
20 turns left
Guess the word:， pro_oun_it_
20 turns left
Guess the word:， pro_oundit_
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
Guess the word:， profoundit_
16 turns

15 turns left
Guess the word:， hallo_ee_
15 turns left
Guess the word:， hallo_een
15 turns left
14 turns left
Guess the word:， halloween
『Hangman』，hints： _______ character :  7
26 turns left
25 turns left
24 turns left
Guess the word:， ___a___
24 turns left
23 turns left
22 turns left
21 turns left
20 turns left
Guess the word:， t__a___
20 turns left
Guess the word:， t_ia_i_
20 turns left
19 turns left
Guess the word:， thia_i_
19 turns left
Guess the word:， thia_in
19 turns left
18 turns left
Guess the word:， thiamin
『Hangman』，hints： _________ character :  9
26 turns left
25 turns left
Guess the word:， ____c____
25 turns left
24 turns left
23 turns left
Guess the word:， ____c__e_
23 turns left
22 turns left
Guess the word:， d___c__ed
22 turns left
Guess the word:， d__ici_ed
22 turns left
Guess the word:， do_ici_ed
22 turns left
21 turns left
20 turns left
Guess the word:， do_iciled
20 turns left
Guess the word:， domiciled
『Hangman』，hints： _______ character :  7
26 turns left
Guess the 

Guess the word:， __eeres_
23 turns left
22 turns left
Guess the word:， __eerest
22 turns left
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
14 turns left
13 turns left
Guess the word:， _ueerest
13 turns left
12 turns left
11 turns left
10 turns left
9 turns left
8 turns left
Guess the word:， queerest
『Hangman』，hints： ________ character :  8
26 turns left
Guess the word:， _______s
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， _r_____s
22 turns left
21 turns left
20 turns left
Guess the word:， _r___i_s
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， _r___ins
15 turns left
Guess the word:， _ro__ins
15 turns left
Guess the word:， groggins
『Hangman』，hints： ______ character :  6
26 turns left
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， ____e_
22 turns left
Guess the word:， ___rer
22 turns left
21 turns left
20 turns lef

21 turns left
Guess the word:， sceneries
『Hangman』，hints： ____________ character :  12
26 turns left
25 turns left
Guess the word:， _c_____c____
25 turns left
Guess the word:， aca____ca___
25 turns left
24 turns left
23 turns left
22 turns left
Guess the word:， aca____call_
22 turns left
Guess the word:， aca___icall_
22 turns left
Guess the word:， acad__icall_
22 turns left
Guess the word:， acade_icall_
22 turns left
Guess the word:， academicall_
22 turns left
Guess the word:， academically
『Hangman』，hints： _____ character :  5
26 turns left
Guess the word:， __s__
26 turns left
25 turns left
24 turns left
23 turns left
Guess the word:， __se_
23 turns left
22 turns left
21 turns left
Guess the word:， __set
21 turns left
20 turns left
19 turns left
18 turns left
17 turns left
16 turns left
15 turns left
Guess the word:， _nset
15 turns left
Guess the word:， onset
『Hangman』，hints： ________ character :  8
26 turns left
25 turns left
24 turns left
Guess the word:， _a______
24 turns left
23 tu

In [107]:
print("平均猜錯次數：", result)

平均猜錯次數： 8.947
